In [ ]:
# ===================================================================
# 1. 라이브러리 임포트 및 API 키 설정 (동일)
# ===================================================================
import os
import urllib.request
import datetime
import json
import time
import ssl # HTTPS 인증서 문제를 우회하기 위해 추가

client_id = 'Kc65Ibtu71R5TLRUF7lL'
client_secret = 'CdT4sLf9Ht' # ⬅️ 여기에 본인의 Secret Key를 입력하세요.
# --- 추가: HTTPS 인증서 검증 비활성화 (전역 설정) ---
# 이 코드는 스크립트 전체에 걸쳐 SSL 검증을 비활성화하여 SSL 오류를 방지합니다.
ssl._create_default_https_context = ssl._create_unverified_context

# ===================================================================
# 2. 기본 함수 정의 (User-Agent 추가)
# ===================================================================
def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)

    # --- 수정된 부분: 하이픈(-)을 하나 제거합니다 ---
    req.add_header("X-Naver-Client-Secret", client_secret)

    # 웹 브라우저인 척 하기 위한 헤더 추가
    req.add_header("User-Agent", "Mozilla/5.0")
    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            return response.read().decode('utf-8')
    except Exception as e:
        print(f"[{datetime.datetime.now()}] API 요청 실패: {e} | URL: {url}")
        return None

# 네이버 이미지 검색 API를 호출하는 함수 (수정 없음)
def getNaverImageSearch(srcText, start, display):
    base = "https://openapi.naver.com/v1/search/image"
    parameters = f"?query={urllib.parse.quote(srcText)}&start={start}&display={display}"
    url = base + parameters
    return getRequestUrl(url)

# ===================================================================
# 3. 이미지 크롤링 실행 (메인 로직 - 대폭 수정)
# ===================================================================
KEYWORDS = ["분짜"]
NUM_IMAGES = 200
SAVE_DIR = "2_naver_crawled_images"

print("🚀 Naver 이미지 크롤링을 시작합니다...")

for keyword in KEYWORDS:
    print(f"\n[INFO] '{keyword}' 이미지 다운로드를 시작합니다...")
    download_path = os.path.join(SAVE_DIR, keyword)
    os.makedirs(download_path, exist_ok=True)

    downloaded_count = 0
    success_count = 0
    start_index = 1
    display_size = 100

    while success_count < NUM_IMAGES:
        print(f"   -> API 요청: 시작 위치 {start_index}")

        json_string = getNaverImageSearch(keyword, start_index, display_size)
        if json_string is None:
            print("   -> API 응답이 없습니다. 다음으로 넘어갑니다.")
            break

        response_dict = json.loads(json_string)

        if 'items' not in response_dict or not response_dict['items']:
            print("   -> 더 이상 검색 결과가 없습니다. 다운로드를 중단합니다.")
            break

        for item in response_dict['items']:
            if success_count >= NUM_IMAGES:
                break

            # --- 파일명 및 확장자 처리 로직 수정 ---
            try:
                image_url = item['link']

                # 1. URL에서 '?' 뒤의 파라미터를 모두 제거
                clean_url = image_url.split('?')[0]

                # 2. 순수한 URL에서 확장자 추출
                file_ext = os.path.splitext(clean_url)[1]

                # 3. 확장자가 없는 경우 (예: 일부 블로그 링크)는 건너뛰거나 기본값 부여
                if not file_ext:
                    # print(f"   -> 확장자가 없는 URL 건너뛰기: {image_url}")
                    # continue
                    file_ext = '.jpg' # 또는 기본 확장자 부여

                # 4. 안전한 파일명 생성
                filename = f"{keyword}_{success_count + 1}{file_ext}"
                save_path = os.path.join(download_path, filename)

                # --- 이미지 다운로드 로직 수정 ---
                # User-Agent 헤더를 포함하여 요청 객체를 다시 만듦
                req = urllib.request.Request(image_url, headers={"User-Agent": "Mozilla/5.0"})
                with urllib.request.urlopen(req) as response, open(save_path, 'wb') as out_file:
                    data = response.read() # 이미지 데이터를 메모리에 읽음
                    out_file.write(data)   # 파일로 저장

                success_count += 1

            except Exception as e:
                print(f"   -> 다운로드 실패: {image_url} | 오류: {type(e).__name__} - {e}")

        start_index += display_size
        time.sleep(0.2)

    print(f"✅ '{keyword}' 이미지 {success_count}장 다운로드 완료! -> 저장 경로: {download_path}")

print("\n🎉 모든 Naver 이미지 크롤링 작업이 완료되었습니다.")

🚀 Naver 이미지 크롤링을 시작합니다...

[INFO] '분짜' 이미지 다운로드를 시작합니다...
   -> API 요청: 시작 위치 1
   -> 다운로드 실패: http://down.humoruniv.com/hwiparambbs/data/pdswait/a_wb2637b001_0517a66e10fb7a22dd63b74445f22d5a3706602d.jpg | 오류: HTTPError - HTTP Error 404: Not Found
